In [10]:
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
import pickle
import copy

import nltk
from nltk.parse import CoreNLPParser
import statistics

from word_freq import preprocess

from collections import Counter

from sklearn import preprocessing

In [1]:
grade = "3"
target_path = f"../../data/question_correctness/.csv"
scores_path = "../../Gates.ReadComp_By-Item_Gr3-5(CM).xlsx"
corpus_path = f"../../subtest_txt/gr{grade}_paragraphs.txt"
questions_path = f"../../subtest_txt/gr{grade}_questions.txt"

### Subtest Embedding

In [3]:
sub_tests = []
embedding_to_
with open(corpus_path,'r') as fp:
    sub_test = fp.readline()
    while sub_test:
        sub_tests.append(sub_test)
        sub_test = fp.readline()


df = pd.read_excel(scores_path)
sub_test_number = df.columns[1:]

print(sub_test_number)

questions_ranges = [(1,5), (6,8), (9,13), (14,16), (17,21), (22,27), (28,30),
                   (31,35), (36,40), (41,43), (44,48)]

model = SentenceTransformer('paraphrase-mpnet-base-v2')

#Change the length to 200
model.max_seq_length = 500

sub_tests_embed = model.encode(sub_tests, show_progress_bar=True)

print(sub_tests_embed.shape)

Index(['Gr3.RC.Gates_01', 'Gr3.RC.Gates_02', 'Gr3.RC.Gates_03',
       'Gr3.RC.Gates_04', 'Gr3.RC.Gates_05', 'Gr3.RC.Gates_06',
       'Gr3.RC.Gates_07', 'Gr3.RC.Gates_08', 'Gr3.RC.Gates_09',
       'Gr3.RC.Gates_10',
       ...
       'Gr5.RC.Gates_45', 'Gr5.RC.Gates_46', 'Gr5.RC.Gates_47',
       'Gr5.RC.Gates_48', 'Gr5.RC.Gates_RawScore', 'Gr5.RC.Gates_GradeEquiv',
       'Gr5.RC.Gates_NCE', 'Gr5.RC.Gates_NPR', 'Gr5.RC.Gates_NS',
       'Gr5.RC.Gates_ESS'],
      dtype='object', length=162)



(12, 768)


### Subtest rare words

In [53]:
sub_tests_rare = []
def get_rare_words_perc(text,wordlist):
    tokens = preprocess(text)
    counter = 0
    for t in tokens:
        if t not in wordlist:
            counter += 1
    return counter/len(tokens)

with open("wordlist.pkl",'rb') as fp:
    wordlist = pickle.load(fp)

for st in sub_tests:
    sub_tests_rare.append(get_rare_words_perc(st, wordlist))
    
print(sub_tests_rare)

[0.3103448275862069, 0.5128205128205128, 0.43902439024390244, 0.45454545454545453, 0.5483870967741935, 0.5441176470588235, 0.55, 0.5757575757575758, 0.5849056603773585, 0.43137254901960786, 0.4473684210526316, 0.5277777777777778]


In [6]:
with open("awl.txt",'r') as file:
    corpus = file.read().replace('\n', ' ')
tokens = preprocess(corpus)
freq = Counter(tokens)
print(freq)
rare_list = freq.keys()

for i in range(len(sub_tests)):
    print(i)
    tokens = preprocess(sub_tests[i])
    counter = 0
    for t in tokens:
        if t in rare_list:
            print(t)
            counter += 1
    print('total:', counter)

Counter({'academic': 11, 'sublist': 9, 'word': 9, 'list': 9, 'nonconformist': 4, 'formula': 3, 'coordinate': 3, 'hypothesis': 3, 'appendix': 3, 'analyser': 2, 'analysis': 2, 'analyst': 2, 'approach': 2, 'area': 2, 'ass': 2, 'assessment': 2, 'assumption': 2, 'authority': 2, 'benefit': 2, 'beneficiary': 2, 'concept': 2, 'inconsistency': 2, 'constituency': 2, 'constituent': 2, 'constitution': 2, 'context': 2, 'contract': 2, 'contractor': 2, 'creation': 2, 'creator': 2, 'definition': 2, 'derivation': 2, 'derivative': 2, 'distribution': 2, 'distributor': 2, 'economy': 2, 'economist': 2, 'environment': 2, 'environmentalist': 2, 'establishment': 2, 'estimate': 2, 'estimation': 2, 'overestimate': 2, 'underestimate': 2, 'export': 2, 'exporter': 2, 'factor': 2, 'finance': 2, 'financier': 2, 'formulation': 2, 'function': 2, 'identity': 2, 'income': 2, 'indication': 2, 'indicator': 2, 'individual': 2, 'individualist': 2, 'interpretation': 2, 'misinterpretation': 2, 'reinterpretation': 2, 'issue': 

In [55]:
get_rare_words_perc(sub_tests[0])

22


### Calculate parse tree depth

In [13]:
def get_average_depth(parser,text):
    sentences = nltk.sent_tokenize(text)
#     print(sentences)
    depths = []
    for s in sentences:
        parse = next(parser.raw_parse(s))
#         parse.draw()
        depths.append(parse.height())
        
    return statistics.mean(depths)

def get_depth_stats(parser,text):
    sentences = nltk.sent_tokenize(text)
#     print(sentences)
    depths = []
    for s in sentences:
        parse = next(parser.raw_parse(s))
#         parse.draw()
        depths.append(parse.height())
        
    return [statistics.mean(depths), statistics.stdev(depths), max(depths)]
#     return [min(depths), max(depths)]

parser = CoreNLPParser(url='http://localhost:9000')

# parse = next(parser.raw_parse(nltk.sent_tokenize(sub_tests[7])[4]))
# print(parse.height())
# parse.draw()



### Questions and Answers Embedding

In [6]:
questions = []
answers = []

with open(questions_path,'r') as fp:
    for _ in range(48):
        question = fp.readline()
        answer = fp.readline()
#         print(question,answer)
        questions.append(question)
        answers.append(answer)
        
print(len(questions))
print(len(answers))

questions_embed = model.encode(questions, show_progress_bar=True)
answers_embed = model.encode(answers, show_progress_bar=True)

print(questions_embed.shape)
print(answers_embed.shape)

48
48



(48, 768)
(48, 768)


### Sentence Complexity

In [15]:
depths = []
questions_depth = []
answer_depth = []

for text in sub_tests:
    depths.append(get_depth_stats(parser,text))

for q in questions:
    questions_depth.append(get_average_depth(parser,q))

for a in answers:
    answer_depth.append(get_average_depth(parser,a))

### Word Frequency

### Feature names

In [35]:
df2 = pd.read_excel(feature_path)

In [48]:
feature_path = "../../SoR_Alberta.Shared.Data.and.Codebook.xlsx"#"../data/gr3/gr3_features.xlsx"
feature_names = ['G3.PPVT.Vocab.raw',
                 'G3.Elision.PA.raw',
                 'G3.Syn.GramCorrect.raw',
                 'G3.DigitSpan.raw',
                 'G3.WordID.raw',
                 'G3.TOWRE.PDE.raw',
                ]


In [45]:
# remove non-full entries
for name in feature_names:
    unavailable_index = df2[df2[name]<0].index
    df2.drop(unavailable_index , inplace=True)

In [46]:
# add towre scores
df2['G3.TOWRE.raw'] = df2['G3.TOWRE.SWE.raw'] + df2['G3.TOWRE.PDE.raw']

In [52]:
for name in feature_names:
    print(name)
    print(df2[name].describe())

# normalize all feature using min-max normalization
normalized_df2=(df2-df2.min())/(df2.max()-df2.min())


G3.PPVT.Vocab.raw
count    138.000000
mean      31.949275
std        4.930647
min       19.000000
25%       29.000000
50%       32.000000
75%       36.000000
max       46.000000
Name: G3.PPVT.Vocab.raw, dtype: float64
G3.Elision.PA.raw
count    138.000000
mean      23.014493
std        6.097727
min        8.000000
25%       17.250000
50%       25.000000
75%       28.000000
max       33.000000
Name: G3.Elision.PA.raw, dtype: float64
G3.Syn.GramCorrect.raw
count    138.000000
mean       9.565217
std        3.266634
min        2.000000
25%        7.000000
50%       10.000000
75%       12.000000
max       16.000000
Name: G3.Syn.GramCorrect.raw, dtype: float64
G3.DigitSpan.raw
count    138.000000
mean      13.173913
std        2.308072
min        8.000000
25%       12.000000
50%       13.000000
75%       15.000000
max       20.000000
Name: G3.DigitSpan.raw, dtype: float64
G3.WordID.raw
count    138.000000
mean      60.971014
std       12.723874
min        3.000000
25%       55.250000
50%   

### Assemble all features into one list

In [55]:
data = []


combined = []

for i in normalized_df2.index:

    skills = []
    # Retrieve all skill feature data
    for name in feature_names:
        value = normalized_df2[name][i]
        skills.append(value)
    
    for index, q_range in enumerate(questions_ranges):
        entry = []
        entry.append(skills+depths[index]+[sub_tests_rare[index]]) ### append depth and rare word feature
        entry.append(sub_tests_embed[index])
        
        for j in range(q_range[0],q_range[1]+1):
            # add embeddings
            detail = []
            detail.append(questions_embed[j-1])
            detail.append(answers_embed[j-1])
            # add cfg depth
            new_entry = copy.deepcopy(entry)

            new_entry[0] += [questions_depth[j-1],answer_depth[j-1]]
            new_entry[0] = np.asarray(new_entry[0])
            
            if df[f"Gr{grade}.RC.Gates_"+"{:02d}".format(j)][i] == 1:
                detail.append(1)
            elif df[f"Gr{grade}.RC.Gates_"+"{:02d}".format(j)][i] in (2,0):
                detail.append(0)
            else:
                print('DNE')
            data.append(new_entry+detail+[index])

print(data[0][0])
print(data[400][5])
    

[ 0.51851852  0.72        0.21428571  0.25        0.7         0.43137255
  7.66666667  1.3662601  10.          0.31034483  7.          4.        ]
4


In [25]:
with open("data.pkl",'wb') as fp:
    pickle.dump(data, fp)


In [66]:
with open("data.pkl",'rb') as fp:
    new_data = pickle.load(fp)

In [67]:
print(len(new_data[1][0]))
# print(len(new_data))

12
